In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.callbacks import ModelCheckpoint

import pickle
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from scipy.optimize import minimize

In [19]:
# BasicCNN
# 284,947 params
def getModel0():
  opt = tf.keras.optimizers.Adam()

  model = keras.Sequential()
  model.add(keras.layers.Conv1D(64, 192, kernel_initializer='normal', activation='relu', input_shape=(1500, 1)))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.MaxPooling1D(5))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Conv1D(48, 64, kernel_initializer='normal', activation='relu'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.MaxPooling1D(5))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Conv1D(32, 18, kernel_initializer='normal', activation='relu'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(64, kernel_initializer='normal', activation='relu'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Dense(32, kernel_initializer='normal', activation='relu'))
  model.add(keras.layers.BatchNormalization())
  model.add(keras.layers.Dense(3, kernel_initializer='normal', activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

model = getModel0()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_205 (Conv1D)         (None, 1309, 64)          12352     
                                                                 
 batch_normalization_228 (Ba  (None, 1309, 64)         256       
 tchNormalization)                                               
                                                                 
 max_pooling1d_56 (MaxPoolin  (None, 261, 64)          0         
 g1D)                                                            
                                                                 
 dropout_90 (Dropout)        (None, 261, 64)           0         
                                                                 
 conv1d_206 (Conv1D)         (None, 198, 48)           196656    
                                                                 
 batch_normalization_229 (Ba  (None, 198, 48)         

In [18]:
# ResNetLikeCNNx3
# 344,259 params
def getModel1():
  opt = tf.keras.optimizers.Adam()
  inputs = tf.keras.Input(shape=(1500, 1))

  x = tf.keras.layers.Conv1D(64, 192, activation='relu')(inputs)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(7)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 1
  shortcut = tf.keras.layers.Conv1D(48, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(48, 32, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(48, 32, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(48, 32, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(5)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 2
  shortcut = tf.keras.layers.Conv1D(24, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(24, 16, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(24, 16, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(24, 16, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(48, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(24, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

model = getModel1()
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 1500, 1)]    0           []                               
                                                                                                  
 conv1d_196 (Conv1D)            (None, 1309, 64)     12352       ['input_15[0][0]']               
                                                                                                  
 batch_normalization_217 (Batch  (None, 1309, 64)    256         ['conv1d_196[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 max_pooling1d_54 (MaxPooling1D  (None, 187, 64)     0           ['batch_normalization_217[

In [17]:
# ResNetLikeCNNx4
# 309,051 params
def getModel2():
  opt = tf.keras.optimizers.Adam()
  inputs = tf.keras.Input(shape=(1500, 1))

  x = tf.keras.layers.Conv1D(64, 192, activation='relu')(inputs)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(7)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 1
  shortcut = tf.keras.layers.Conv1D(48, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(96, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(96, 9, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(96, 9, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(48, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(5)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 2
  shortcut = tf.keras.layers.Conv1D(24, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(72, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(72, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(72, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(24, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(48, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(24, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

model = getModel2()
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 1500, 1)]    0           []                               
                                                                                                  
 conv1d_185 (Conv1D)            (None, 1309, 64)     12352       ['input_14[0][0]']               
                                                                                                  
 batch_normalization_204 (Batch  (None, 1309, 64)    256         ['conv1d_185[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 max_pooling1d_52 (MaxPooling1D  (None, 187, 64)     0           ['batch_normalization_204[

In [16]:
# SmallerResNet4x2
# 147,811 params
def getModel3():
  opt = tf.keras.optimizers.Adam()
  inputs = tf.keras.Input(shape=(1500, 1))

  x = tf.keras.layers.Conv1D(32, 15, padding='same', activation='relu')(inputs)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(5)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 1
  shortcut = tf.keras.layers.Conv1D(48, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(48, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(48, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 2
  shortcut = tf.keras.layers.Conv1D(56, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(56, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(56, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 3
  shortcut = tf.keras.layers.Conv1D(64, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 4
  shortcut = tf.keras.layers.Conv1D(72, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(72, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(72, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.Dropout(0.5)(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(40, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

model = getModel3()
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 1500, 1)]    0           []                               
                                                                                                  
 conv1d_172 (Conv1D)            (None, 1500, 32)     512         ['input_13[0][0]']               
                                                                                                  
 batch_normalization_190 (Batch  (None, 1500, 32)    128         ['conv1d_172[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 max_pooling1d_48 (MaxPooling1D  (None, 300, 32)     0           ['batch_normalization_190[

In [15]:
# SmallerResNet4x3
# 151,451 params
def getModel4():
  opt = tf.keras.optimizers.Adam()
  inputs = tf.keras.Input(shape=(1500, 1))

  x = tf.keras.layers.Conv1D(32, 15, padding='same', activation='relu')(inputs)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(5)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 1
  shortcut = tf.keras.layers.Conv1D(40, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(48, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(48, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(40, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 2
  shortcut = tf.keras.layers.Conv1D(48, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(64, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(48, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 3
  shortcut = tf.keras.layers.Conv1D(56, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(74, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(74, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(56, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 4
  shortcut = tf.keras.layers.Conv1D(64, 1)(x)
  shortcut = tf.keras.layers.BatchNormalization()(shortcut)

  x = tf.keras.layers.Conv1D(96, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(96, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(64, 1, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.Dropout(0.5)(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(48, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

model = getModel4()
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 1500, 1)]    0           []                               
                                                                                                  
 conv1d_155 (Conv1D)            (None, 1500, 32)     512         ['input_12[0][0]']               
                                                                                                  
 batch_normalization_172 (Batch  (None, 1500, 32)    128         ['conv1d_155[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 max_pooling1d_44 (MaxPooling1D  (None, 300, 32)     0           ['batch_normalization_172[

In [4]:
# IdentityResNet4x2
#  params
def getModel5():
  opt = tf.keras.optimizers.Adam()
  inputs = tf.keras.Input(shape=(1500, 1))

  x = tf.keras.layers.Conv1D(64, 15, padding='same', activation='relu')(inputs)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(5)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 1
  shortcut = x

  x = tf.keras.layers.Conv1D(64, 7, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(64, 7, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 2
  shortcut = x

  x = tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 3
  shortcut = x

  x = tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.MaxPooling1D(3)(x)
  x = tf.keras.layers.Dropout(0.5)(x)

  # Residual block 4
  shortcut = x

  x = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Add()([x, shortcut])

  x = tf.keras.layers.Dropout(0.5)(x)

  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  x = tf.keras.layers.Dense(48, activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)

  outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

  return model

model = getModel5()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1500, 1)]    0           []                               
                                                                                                  
 conv1d_18 (Conv1D)             (None, 1500, 64)     1024        ['input_3[0][0]']                
                                                                                                  
 batch_normalization_20 (BatchN  (None, 1500, 64)    256         ['conv1d_18[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 max_pooling1d_8 (MaxPooling1D)  (None, 300, 64)     0           ['batch_normalization_20[0]